In [ ]:
!pip install -q pytorchvideo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.7/132.7 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.0/31.0 MB 55.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:

import pytorchvideo.data

from pytorchvideo.transforms import (
    ApplyTransformToKey,
    Normalize,
    RandomShortSideScale,
    RemoveKey,
    ShortSideScale,
    UniformTemporalSubsample,
)

from torchvision.transforms import (
    Compose,
    Lambda,
    RandomCrop,
    RandomHorizontalFlip,
    RandomPerspective,
    Pad,
    Resize,
)

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional_tensor.py:5: UserWarning: The torchvision.transforms.functional_tensor module is deprecated in 0.15 and will be **removed in 0.17**. Please don't rely on it. You probably just need to use APIs in torchvision.transforms.functional or in torchvision.transforms.v2.functional.
  warnings.warn(


In [ ]:
import random
import numpy as np
import os
import torch
import torchvision
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)
os.environ['PYTHONHASHSEED'] = str(42)

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 61.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.6 MB/s eta 0:00:00


In [ ]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 21.4 MB/s eta 0:00:00


In [ ]:
import subprocess
import cv2
def fps_sec_vid(path):
    cam = cv2.VideoCapture(path)
    fps = cam.get(cv2.CAP_PROP_FPS)
    result = subprocess.run(["ffprobe", "-v", "error", "-show_entries",
                                 "format=duration", "-of",
                                 "default=noprint_wrappers=1:nokey=1", path],
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT)
    sec=float(result.stdout)
    return fps, sec

In [ ]:
import torch
from PIL import Image
import av
import numpy as np
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


def read_video_pyav(container, indices):
    '''
    Decode the video with PyAV decoder.
    Args:
        container (`av.container.input.InputContainer`): PyAV container.
        indices (`List[int]`): List of frame indices to decode.
    Returns:
        result (np.ndarray): np array of decoded frames of shape (num_frames, height, width, 3).
    '''
    frames = []
    container.seek(0)
    start_index = indices[0]
    end_index = indices[-1]
    for i, frame in enumerate(container.decode(video=0)):
        if i > end_index:
            break
        if i >= start_index and i in indices:
            frames.append(frame)
    return np.stack([x.to_ndarray(format="rgb24") for x in frames])


def sample_frame_indices(clip_len, frame_sample_rate, seg_len):
    '''
    Sample a given number of frame indices from the video.
    Args:
        clip_len (`int`): Total number of frames to sample.
        frame_sample_rate (`int`): Sample every n-th frame.
        seg_len (`int`): Maximum allowed index of sample's last frame.
    Returns:
        indices (`List[int]`): List of sampled frame indices
    '''
    converted_len = int(clip_len * frame_sample_rate)
    end_idx = np.random.randint(converted_len, seg_len)
    start_idx = end_idx - converted_len
    indices = np.linspace(start_idx, end_idx, num=clip_len)
    indices = np.clip(indices, start_idx, end_idx - 1).astype(np.int64)
    return indices


# video clip consists of 300 frames (10 seconds at 30 FPS)
# file_path = hf_hub_download(
#     repo_id="nielsr/video-demo", filename="eating_spaghetti.mp4", repo_type="dataset"
# )
container = av.open('/content/drive/MyDrive/cyberpunkedgerunners_trailer2_1920.mp4')
fps, secs = fps_sec_vid('/content/drive/MyDrive/cyberpunkedgerunners_trailer2_1920.mp4')

# sample 16 frames
indices = sample_frame_indices(clip_len=secs, frame_sample_rate=fps, seg_len=container.streams.video[0].frames)
video = read_video_pyav(container, indices)

In [ ]:
from transformers import VisionEncoderDecoderModel, ViTFeatureExtractor, AutoTokenizer
import torch
from PIL import Image

model = VisionEncoderDecoderModel.from_pretrained("tuman/vit-rugpt2-image-captioning")
feature_extractor = ViTFeatureExtractor.from_pretrained("tuman/vit-rugpt2-image-captioning")
tokenizer = AutoTokenizer.from_pretrained("tuman/vit-rugpt2-image-captioning")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

max_length = 60
num_beams = 4
gen_kwargs = {"max_length": max_length, "num_beams": num_beams}

def predict_caption(video):
  predss = []
  for image in video:
    pixel_values = feature_extractor(images=[image], return_tensors="pt").pixel_values
    pixel_values = pixel_values.to(device)

    output_ids = model.generate(pixel_values, **gen_kwargs)

    preds = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
    print(preds)
    predss.append([pred.strip() for pred in preds])
    del preds
    del pixel_values
    del output_ids
  return predss

predict_caption(video)


/usr/local/lib/python3.10/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


['Оживленная городская улица ночью с множеством огней.']
['Улыбающаяся женщина держит табличку с надписью «Я не могу видеть изображение выше».']
['Черно-белое фото человека, сидящего на полу.']
['Стол, на котором лежат разные предметы.']
['Стол, на котором лежат разные предметы.']
['Стол, на котором лежат разные предметы.']
['Женщина сидит за столом с ноутбуком.']
['Женщина, сидящая на скамейке, смотрит в мобильный телефон.']
['Женщина, сидящая в кресле, разговаривает по мобильному телефону.']
['Черно-белое фото человека, держащего мобильный телефон.']
['Черно-белая фотография человека, держащего мобильный телефон.']
['Две фотографии человека, держащего два разных предмета.']
['Стеклянная скульптура слона, сидящего на столе.']
['Стеклянная скульптура слона, сидящего на столе.']
['Черно-белая фотография человека, держащего табличку.']
['Черно-белая фотография человека, держащего мобильный телефон.']
['Крупный план трех разных изображений.']
['Две фотографии человека, держащего сноуборд.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-01f316406950>", line 32, in <cell line: 32>
    predict_caption(video)
  File "<ipython-input-2-01f316406950>", line 22, in predict_caption
    output_ids = model.generate(pixel_values, **gen_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/_contextlib.py", line 115, in decorate_context
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py", line 1681, in generate
    return self.beam_search(
  File "/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py", line 3020, in beam_search
    outputs = self(
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1501, in _call_impl
    return forward_call(*args, **kwargs)
  File "/usr/local/lib/python3.10

TypeError: ignored

In [ ]:
import gc
gc.collect()

622

In [ ]:
len(video)

132